In [1]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00


In [2]:
!wget -N -q "https://github.com/thomasfork/ipopt_linux/raw/main/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [3]:
import numpy as np
from pyomo.environ import *

In [4]:
def styblinski_tang(model):
  delta = 0.
  for i in model.x:
    delta = delta + model.params[1] * (model.params[2] * model.x[i] * model.x[i] * model.x[i] * model.x[i] + model.params[3] * model.x[i]**2 + model.params[4] * model.x[i])
  return delta

In [5]:
# --- Create a model
model               = ConcreteModel()

N_unknowns          = 10
model.N_unknowns    = Param(RangeSet(1, 1), initialize = N_unknowns, within = Integers)
model.i             = RangeSet(1, N_unknowns)
model.x             = Var(model.i, within = Reals)
v                   = {}
v[1]                = 0.5
v[2]                = 1.
v[3]                = -16.
v[4]                = 5.
model.params        = Param(RangeSet(1, 4), initialize = v, within = Reals)
for i in model.i:
  model.x[i].value = -.3
  model.x[i].lower = -5.
  model.x[i].upper = 5.

# --- Declare objective function
model.objfun    = Objective(rule = styblinski_tang, sense = minimize)

model.pprint()

1 RangeSet Declarations
    i : Dimen=1, Size=10, Bounds=(1, 10)
        Key  : Finite : Members
        None :   True :  [1:10]

2 Param Declarations
    N_unknowns : Size=1, Index=[1], Domain=Integers, Default=None, Mutable=False
        Key : Value
          1 :    10
    params : Size=4, Index=[1:4], Domain=Reals, Default=None, Mutable=False
        Key : Value
          1 :   0.5
          2 :   1.0
          3 : -16.0
          4 :   5.0

1 Var Declarations
    x : Size=10, Index=i
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  -5.0 :  -0.3 :   5.0 : False : False :  Reals
          2 :  -5.0 :  -0.3 :   5.0 : False : False :  Reals
          3 :  -5.0 :  -0.3 :   5.0 : False : False :  Reals
          4 :  -5.0 :  -0.3 :   5.0 : False : False :  Reals
          5 :  -5.0 :  -0.3 :   5.0 : False : False :  Reals
          6 :  -5.0 :  -0.3 :   5.0 : False : False :  Reals
          7 :  -5.0 :  -0.3 :   5.0 : False : False :  Reals
          8 :  -5.0

In [6]:
SolverFactory('ipopt', executable='/content/ipopt').solve(model).write()
#SolverFactory('multistart').solve(model, strategy = 'rand_guess_and_bound', solver = 'ipopt', iterations = 50).write()

# display solution
print('\nObjective function = ', model.objfun)

print('\nDecision Variables')
for i in model.x:
  print(model.x[i]())

print(model.N_unknowns[1])

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 10
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.03781867027282715
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objec